In [1]:
import torch

In [2]:
A = torch.Tensor([
    [0, 1, 0, 10, 100, 0],
    [1, 0, 0, 10, 0, 0]
])

embeddings = torch.Tensor([
    [0, 1, 1,],
    [2, 3, 3],
    [4, 5, 5],
    [6, 7, 7],
    [8 ,9, 9],
])

def add_pad_embedding(embeddings):
    return torch.cat([embeddings, torch.zeros(1, embeddings.shape[1])])

embeddings = add_pad_embedding(embeddings)
embeddings = torch.nn.Embedding.from_pretrained(embeddings, freeze=False, padding_idx=-1)
display(embeddings)
display(embeddings.weight)
display(A)

Embedding(6, 3, padding_idx=5)

Parameter containing:
tensor([[0., 1., 1.],
        [2., 3., 3.],
        [4., 5., 5.],
        [6., 7., 7.],
        [8., 9., 9.],
        [0., 0., 0.]], requires_grad=True)

tensor([[  0.,   1.,   0.,  10., 100.,   0.],
        [  1.,   0.,   0.,  10.,   0.,   0.]])

In [3]:
'''
1. turn nonzeros into indices
2. use indices as in normal transformer to get embedding,
pretty much like token indices - should I include padding tokens?
3. but also somehow use these indices to extract the output
4. it may be that I can also accomplish this via masking
'''
def pad(vec, seq_len, pad_token=-1):
    return torch.cat([
        vec, torch.Tensor([pad_token]).expand(seq_len - vec.size()[0])
    ])

def select_nonzero(A, seq_len=10, pad_token=-1):
    seqs = []
    for row in A:
        nonzeros = pad(row.nonzero().T[0], seq_len, pad_token)
        seqs.append(nonzeros)
    return torch.stack(seqs)

def select_nonzero_vec(seq, seq_len=10, pad_token=-1):
    return seq.gather(1, (seq == 0).sort(dim=1, stable=True)[1])

def index_tensor(seq, indices):
    return seq.T[indices.long()].permute(0,2,1).diagonal().T

In [4]:
display(A)
A.nonzero()

tensor([[  0.,   1.,   0.,  10., 100.,   0.],
        [  1.,   0.,   0.,  10.,   0.,   0.]])

tensor([[0, 1],
        [0, 3],
        [0, 4],
        [1, 0],
        [1, 3]])

In [5]:
nonzeros = select_nonzero(A)
nonzeros_vec = select_nonzero_vec(A)
display(A)
display(nonzeros)
display(nonzeros_vec)

tensor([[  0.,   1.,   0.,  10., 100.,   0.],
        [  1.,   0.,   0.,  10.,   0.,   0.]])

tensor([[ 1.,  3.,  4., -1., -1., -1., -1., -1., -1., -1.],
        [ 0.,  3., -1., -1., -1., -1., -1., -1., -1., -1.]])

tensor([[  1.,  10., 100.,   0.,   0.,   0.],
        [  1.,  10.,   0.,   0.,   0.,   0.]])

In [6]:
A

tensor([[  0.,   1.,   0.,  10., 100.,   0.],
        [  1.,   0.,   0.,  10.,   0.,   0.]])

In [7]:
nonzeros

tensor([[ 1.,  3.,  4., -1., -1., -1., -1., -1., -1., -1.],
        [ 0.,  3., -1., -1., -1., -1., -1., -1., -1., -1.]])

In [8]:
index_attempt = A.T[nonzeros.long()]

In [9]:
index_attempt.permute(0,2,1).diagonal().T

tensor([[  1.,  10., 100.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  1.,  10.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [10]:
index_tensor(A, nonzeros)

tensor([[  1.,  10., 100.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  1.,  10.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [11]:
nonzeros = select_nonzero(A, pad_token=embeddings.weight.size()[0]-1)
nonzeros

tensor([[1., 3., 4., 5., 5., 5., 5., 5., 5., 5.],
        [0., 3., 5., 5., 5., 5., 5., 5., 5., 5.]])

In [12]:
squished_embeddings = embeddings(nonzeros.long())
squished_embeddings

tensor([[[2., 3., 3.],
         [6., 7., 7.],
         [8., 9., 9.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 1., 1.],
         [6., 7., 7.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], grad_fn=<EmbeddingBackward0>)

In [13]:
squished_embeddings.shape

torch.Size([2, 10, 3])

In [14]:
display(squished_embeddings)
display(nonzeros)

tensor([[[2., 3., 3.],
         [6., 7., 7.],
         [8., 9., 9.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 1., 1.],
         [6., 7., 7.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]], grad_fn=<EmbeddingBackward0>)

tensor([[1., 3., 4., 5., 5., 5., 5., 5., 5., 5.],
        [0., 3., 5., 5., 5., 5., 5., 5., 5., 5.]])

In [15]:
squished_embeddings * index_tensor(A, nonzeros).unsqueeze(-1)

tensor([[[  2.,   3.,   3.],
         [ 60.,  70.,  70.],
         [800., 900., 900.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]],

        [[  0.,   1.,   1.],
         [ 60.,  70.,  70.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.],
         [  0.,   0.,   0.]]], grad_fn=<MulBackward0>)

In [16]:
squished_embeddings * nonzeros.unsqueeze(-1)

tensor([[[ 2.,  3.,  3.],
         [18., 21., 21.],
         [32., 36., 36.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]],

        [[ 0.,  0.,  0.],
         [18., 21., 21.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]]], grad_fn=<MulBackward0>)

In [17]:
squished_embeddings.shape

torch.Size([2, 10, 3])